In [1]:
# Data manipulation
import numpy as np
import pandas as pd
pd.options.display.max_rows = 100

# Recommender systems
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split
import surprise as sp

# Other
import os
import random
import sys

# Reload imported code
%load_ext autoreload
%autoreload 2

# Custom modules
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from src import BaselineModel, MatrixFactorization, train_update_test_split

rand_seed = 2
np.random.seed(rand_seed)
random.seed(rand_seed)

## Load data

**Movie data found here https://grouplens.org/datasets/movielens/**

In [2]:
cols = ['user_id', 'item_id', 'rating', 'timestamp']
movie_data = pd.read_csv('../data/ml-1m/ratings.dat', names = cols, sep = '::', usecols=[0, 1, 2], engine='python')

# Prepare data
train, test = train_test_split(movie_data, test_size=0.2)

# Prepare data for online learning
train_initial, train_update, test_update = train_update_test_split(movie_data, frac_new_users=0.2)

## Simple model with global mean

This is essentially just the global standard deviation

In [3]:
global_mean = train['rating'].mean()
pred = [global_mean for _ in range(test.shape[0])]

rmse = mean_squared_error(test['rating'], pred, squared = False)

print(f'\nTest RMSE: {rmse:4f}')

print(f"\nOr equivalently the standard deviation: {train['rating'].std()}")


Test RMSE: 1.117284

Or equivalently the standard deviation: 1.117056401449951


## Baseline Model with biases

### SGD

In [4]:
%%time

baseline_model = BaselineModel(n_epochs = 20, reg = 0.005, verbose=1)
baseline_model.fit(train)

pred = baseline_model.predict(test)
rmse = mean_squared_error(test['rating'], pred, squared = False)

print(f'\nTest RMSE: {rmse:.4f}')

Epoch  1 / 20  -  train_rmse: 0.9160605268044618
Epoch  2 / 20  -  train_rmse: 0.9056052248600731
Epoch  3 / 20  -  train_rmse: 0.9022922945725866
Epoch  4 / 20  -  train_rmse: 0.9008197890805594
Epoch  5 / 20  -  train_rmse: 0.90004570136268
Epoch  6 / 20  -  train_rmse: 0.8995941283431694
Epoch  7 / 20  -  train_rmse: 0.8993105442047119
Epoch  8 / 20  -  train_rmse: 0.899122047751058
Epoch  9 / 20  -  train_rmse: 0.8989908337794819
Epoch  10 / 20  -  train_rmse: 0.8988958744519882
Epoch  11 / 20  -  train_rmse: 0.898824815825671
Epoch  12 / 20  -  train_rmse: 0.8987700702794992
Epoch  13 / 20  -  train_rmse: 0.8987268014078751
Epoch  14 / 20  -  train_rmse: 0.8986918269548304
Epoch  15 / 20  -  train_rmse: 0.898662994107046
Epoch  16 / 20  -  train_rmse: 0.8986388100197431
Epoch  17 / 20  -  train_rmse: 0.898618216087896
Epoch  18 / 20  -  train_rmse: 0.8986004461025785
Epoch  19 / 20  -  train_rmse: 0.898584934890502
Epoch  20 / 20  -  train_rmse: 0.898571258162793

Test RMSE: 0.912

### ALS

In [5]:
%%time

baseline_model = BaselineModel(method='als', n_epochs = 20, reg = 0.5, verbose=1)
baseline_model.fit(train)

pred = baseline_model.predict(test)
rmse = mean_squared_error(test['rating'], pred, squared = False)

print(f'\nTest RMSE: {rmse:.4f}')

Epoch  1 / 20  -  train_rmse: 0.9176617885927044
Epoch  2 / 20  -  train_rmse: 0.8971604837483708
Epoch  3 / 20  -  train_rmse: 0.8961942712223132
Epoch  4 / 20  -  train_rmse: 0.8961360919877193
Epoch  5 / 20  -  train_rmse: 0.8961314250532659
Epoch  6 / 20  -  train_rmse: 0.8961308182120774
Epoch  7 / 20  -  train_rmse: 0.8961306848554272
Epoch  8 / 20  -  train_rmse: 0.8961306360024888
Epoch  9 / 20  -  train_rmse: 0.8961306058768967
Epoch  10 / 20  -  train_rmse: 0.896130580306678
Epoch  11 / 20  -  train_rmse: 0.8961305559767064
Epoch  12 / 20  -  train_rmse: 0.8961305320922546
Epoch  13 / 20  -  train_rmse: 0.8961305084604123
Epoch  14 / 20  -  train_rmse: 0.8961304850332851
Epoch  15 / 20  -  train_rmse: 0.8961304617980095
Epoch  16 / 20  -  train_rmse: 0.8961304387501804
Epoch  17 / 20  -  train_rmse: 0.8961304158874763
Epoch  18 / 20  -  train_rmse: 0.8961303932080733
Epoch  19 / 20  -  train_rmse: 0.8961303707102787
Epoch  20 / 20  -  train_rmse: 0.8961303483924242

Test RMSE

### Updating with new users

In [6]:
baseline_model = BaselineModel(method='sgd', n_epochs = 20, lr=0.01, reg = 0.05, verbose=1)
baseline_model.fit(train_initial)

user_biases = baseline_model.user_biases

Epoch  1 / 20  -  train_rmse: 0.917957622300513
Epoch  2 / 20  -  train_rmse: 0.9063452473554275
Epoch  3 / 20  -  train_rmse: 0.9031838358128945
Epoch  4 / 20  -  train_rmse: 0.9018213344663989
Epoch  5 / 20  -  train_rmse: 0.9011136211672972
Epoch  6 / 20  -  train_rmse: 0.9007003958585306
Epoch  7 / 20  -  train_rmse: 0.9004377099085408
Epoch  8 / 20  -  train_rmse: 0.9002592687764506
Epoch  9 / 20  -  train_rmse: 0.9001313986781218
Epoch  10 / 20  -  train_rmse: 0.9000356850623226
Epoch  11 / 20  -  train_rmse: 0.8999614423500142
Epoch  12 / 20  -  train_rmse: 0.8999021518801612
Epoch  13 / 20  -  train_rmse: 0.8998536600787451
Epoch  14 / 20  -  train_rmse: 0.8998132163090632
Epoch  15 / 20  -  train_rmse: 0.8997789351870569
Epoch  16 / 20  -  train_rmse: 0.899749483990946
Epoch  17 / 20  -  train_rmse: 0.8997238944171748
Epoch  18 / 20  -  train_rmse: 0.899701445537573
Epoch  19 / 20  -  train_rmse: 0.8996815888605382
Epoch  20 / 20  -  train_rmse: 0.8996638990334523


In [7]:
%%time
baseline_model.update_users(train_update, n_epochs=20, lr=0.001, verbose=1)
pred = baseline_model.predict(test_update)
rmse = mean_squared_error(test_update['rating'], pred, squared = False)

print(f'\nTest RMSE: {rmse:.4f}')

Epoch  1 / 20  -  train_rmse: 0.9663118338095819
Epoch  2 / 20  -  train_rmse: 0.9527029131858415
Epoch  3 / 20  -  train_rmse: 0.9440781749790527
Epoch  4 / 20  -  train_rmse: 0.9381602217930907
Epoch  5 / 20  -  train_rmse: 0.9338601056670158
Epoch  6 / 20  -  train_rmse: 0.930598987901338
Epoch  7 / 20  -  train_rmse: 0.9280424557723291
Epoch  8 / 20  -  train_rmse: 0.9259846714487712
Epoch  9 / 20  -  train_rmse: 0.9242925121637688
Epoch  10 / 20  -  train_rmse: 0.9228763313930977
Epoch  11 / 20  -  train_rmse: 0.9216736567787333
Epoch  12 / 20  -  train_rmse: 0.9206396453001708
Epoch  13 / 20  -  train_rmse: 0.9197412683231396
Epoch  14 / 20  -  train_rmse: 0.9189536463105857
Epoch  15 / 20  -  train_rmse: 0.9182576674000171
Epoch  16 / 20  -  train_rmse: 0.9176383969553337
Epoch  17 / 20  -  train_rmse: 0.9170839883423071
Epoch  18 / 20  -  train_rmse: 0.9165849196853852
Epoch  19 / 20  -  train_rmse: 0.9161334478156705
Epoch  20 / 20  -  train_rmse: 0.9157232102002919

Test RMSE

## Matrix Factorization

In [8]:
%%time 
matrix_fact = MatrixFactorization(n_epochs = 20, n_factors = 100, verbose = 1, lr = 0.001, reg = 0.005)
matrix_fact.fit(train)

pred = matrix_fact.predict(test)
rmse = mean_squared_error(test['rating'], pred, squared = False)

print(f'\nTest RMSE: {rmse:.4f}')

Epoch  1 / 20  -  train_rmse: 1.0130595708059675
Epoch  2 / 20  -  train_rmse: 0.9736301815027829
Epoch  3 / 20  -  train_rmse: 0.9523725166773437
Epoch  4 / 20  -  train_rmse: 0.9387101848933337
Epoch  5 / 20  -  train_rmse: 0.9289324269879647
Epoch  6 / 20  -  train_rmse: 0.9214137266030734
Epoch  7 / 20  -  train_rmse: 0.9153311381743325
Epoch  8 / 20  -  train_rmse: 0.9102210065253193
Epoch  9 / 20  -  train_rmse: 0.905799748609798
Epoch  10 / 20  -  train_rmse: 0.9018823390583373
Epoch  11 / 20  -  train_rmse: 0.8983416079178681
Epoch  12 / 20  -  train_rmse: 0.8950863153079484
Epoch  13 / 20  -  train_rmse: 0.8920485764771771
Epoch  14 / 20  -  train_rmse: 0.8891762616549006
Epoch  15 / 20  -  train_rmse: 0.8864281961059287
Epoch  16 / 20  -  train_rmse: 0.8837710152946995
Epoch  17 / 20  -  train_rmse: 0.8811770416835768
Epoch  18 / 20  -  train_rmse: 0.8786228176904035
Epoch  19 / 20  -  train_rmse: 0.8760880763671156
Epoch  20 / 20  -  train_rmse: 0.8735550153781546

Test RMSE

### Updating with new users

In [9]:
matrix_fact = MatrixFactorization(n_epochs = 20, n_factors = 100, verbose = 1, lr = 0.001, reg = 0.5)
matrix_fact.fit(train_initial)

Epoch  1 / 20  -  train_rmse: 1.013600881644264
Epoch  2 / 20  -  train_rmse: 0.9810877273260235
Epoch  3 / 20  -  train_rmse: 0.9650703504949346
Epoch  4 / 20  -  train_rmse: 0.9555490886915851
Epoch  5 / 20  -  train_rmse: 0.9492481848863172
Epoch  6 / 20  -  train_rmse: 0.9447772063597116
Epoch  7 / 20  -  train_rmse: 0.9414460807132315
Epoch  8 / 20  -  train_rmse: 0.9388734404014725
Epoch  9 / 20  -  train_rmse: 0.936831205616552
Epoch  10 / 20  -  train_rmse: 0.9351744797049139
Epoch  11 / 20  -  train_rmse: 0.9338067187946515
Epoch  12 / 20  -  train_rmse: 0.9326610682160079
Epoch  13 / 20  -  train_rmse: 0.9316897364879861
Epoch  14 / 20  -  train_rmse: 0.9308576367575886
Epoch  15 / 20  -  train_rmse: 0.9301384200286485
Epoch  16 / 20  -  train_rmse: 0.9295119112272232
Epoch  17 / 20  -  train_rmse: 0.9289624005826117
Epoch  18 / 20  -  train_rmse: 0.9284774742870956
Epoch  19 / 20  -  train_rmse: 0.9280471952987187
Epoch  20 / 20  -  train_rmse: 0.9276635174495363


MatrixFactorization(lr=0.001, n_epochs=20, reg=0.5)

In [10]:
%%time
matrix_fact.update_users(train_update, lr=0.001, n_epochs=20, verbose=1)
pred = matrix_fact.predict(test_update)
rmse = mean_squared_error(test_update['rating'], pred, squared = False)

print(f'\nTest RMSE: {rmse:.4f}')

Epoch  1 / 20  -  train_rmse: 0.9878377903377871
Epoch  2 / 20  -  train_rmse: 0.9755328040280514
Epoch  3 / 20  -  train_rmse: 0.967897803944037
Epoch  4 / 20  -  train_rmse: 0.96270497883029
Epoch  5 / 20  -  train_rmse: 0.958946071799832
Epoch  6 / 20  -  train_rmse: 0.956099328365387
Epoch  7 / 20  -  train_rmse: 0.9538684251537063
Epoch  8 / 20  -  train_rmse: 0.9520728645290876
Epoch  9 / 20  -  train_rmse: 0.9505964571331216
Epoch  10 / 20  -  train_rmse: 0.9493610139884009
Epoch  11 / 20  -  train_rmse: 0.9483119821166791
Epoch  12 / 20  -  train_rmse: 0.9474101456391137
Epoch  13 / 20  -  train_rmse: 0.9466265960514788
Epoch  14 / 20  -  train_rmse: 0.9459395585645287
Epoch  15 / 20  -  train_rmse: 0.9453323207173054
Epoch  16 / 20  -  train_rmse: 0.9447918412209358
Epoch  17 / 20  -  train_rmse: 0.9443077924670416
Epoch  18 / 20  -  train_rmse: 0.9438718872706293
Epoch  19 / 20  -  train_rmse: 0.9434773964045834
Epoch  20 / 20  -  train_rmse: 0.9431187969062885

Test RMSE: 0.

## Surprise

In [11]:
# Load the movielens-100k dataset (download it if needed),
data = sp.Dataset.load_builtin('ml-100k')

In [12]:
%%time
# We'll use the famous SVD algorithm.
algo = sp.SVD()

# Run 5-fold cross-validation and print results
sp.model_selection.cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9397  0.9355  0.9292  0.9388  0.9341  0.9355  0.0037  
MAE (testset)     0.7399  0.7374  0.7332  0.7386  0.7353  0.7369  0.0024  
Fit time          4.25    4.16    4.22    4.22    4.23    4.22    0.03    
Test time         0.17    0.12    0.11    0.16    0.12    0.14    0.02    
Wall time: 22.4 s


{'test_rmse': array([0.93967479, 0.93553074, 0.92923385, 0.93884081, 0.93411477]),
 'test_mae': array([0.73994964, 0.73736547, 0.73319142, 0.738612  , 0.73534611]),
 'fit_time': (4.249607563018799,
  4.164863586425781,
  4.2217137813568115,
  4.221713066101074,
  4.229688882827759),
 'test_time': (0.16755151748657227,
  0.12466549873352051,
  0.11167263984680176,
  0.16453123092651367,
  0.12067866325378418)}

In [13]:
%%time

# surprise_svd = sp.SVD(lr_all=0.001, reg_all=0.005, n_epochs=20, n_factors=50)
surprise_svd = sp.BaselineOnly()

reader = sp.Reader()

data_train = sp.Dataset.load_from_df(train[['user_id', 'item_id', 'rating']], reader = reader).build_full_trainset()
data_test = sp.Dataset.load_from_df(test[['user_id', 'item_id', 'rating']], reader = reader).build_full_trainset().build_testset()
surprise_svd.fit(data_train)

pred = surprise_svd.test(data_test)
sp.accuracy.rmse(pred)

Estimating biases using als...
RMSE: 0.9106
Wall time: 6.44 s


0.9106051680456124